<a href="https://colab.research.google.com/github/akrysm/rnaseq/blob/main/RNA_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#準備(GoogleDrive, インストール, ディレクトリ・フォルダ設定, コンテンツ確認)

In [ ]:
#@title Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title インストール
import gzip
import locale
import os
import shutil
import subprocess
import time

!apt-get -qq install hisat2
!apt-get -qq install samtools
!apt-get -qq install subread
!apt-get -qq install tree
os.environ['PATH'] += ":/usr/local/bin"
!hisat2 --version
!samtools --version
!subread --version
!tree --version

Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../0-libhtscodecs2_1.1.1-3_amd64.deb ...
Unpacking libhtscodecs2:amd64 (1.1.1-3) ...
Selecting previously unselected package libhts3:amd64.
Preparing to unpack .../1-libhts3_1.13+ds-2build1_amd64.deb ...
Unpacking libhts3:amd64 (1.13+ds-2build1) ...
Selecting previously unselected package bcftools.
Preparing to unpack .../2-bcftools_1.13-1_amd64.deb ...
Unpacking bcftools (1.13-1) ...
Selecting previously unselected package hisat2.
Preparing to unpack .../3-hisat2_2.2.1-3_amd64.deb ...
Unpacking hisat2 (2.2.1-3) ...
Selecting previously unselected package python3-hisat2.
Preparing to unpack .../4-python3-hisat2_2.2.1-3_all.deb ...
Unpacking python3-hisat2 (2.2.1-3) ...
Selecting previously unselected package samtools.
Preparing to unpack .../5-samtools_1.13-4_amd64.deb ...
Unpacking samtools (1.13-4) ...
Setting up libhtscodecs2:amd6

In [ ]:
#@title フォルダ作成: MyDrive以下に [RNAseq] を作成
import os

# working directory path
DIR_WORK = "/content/drive/MyDrive" #@param ["/content/drive/MyDrive", "option"] {allow-input: true}

# %cd {DIR_RAWDATA}  # linux
os.chdir(DIR_WORK)
print("現在のディレクトリ:", os.getcwd())

# folder name
directories = ["RNAseq"]

# create folder function
def check_and_create_directory(directory):
    if not os.path.exists(directory):
        print(f"Creating directory: {directory}")
        os.makedirs(directory)
    else:
        print(f"Directory already exists: {directory}. Nothing to do.")

# create folder
for directory in directories:
    check_and_create_directory(directory)

In [ ]:
#@title フォルダ作成: RNAseq以下に [bamfiles, downloads, featureCounts, multiqc, rawdata, trimmed] を作成
import os

# working directory path
DIR_WORK = "/content/drive/MyDrive/RNAseq" #@param ["/content", "/content/drive/MyDrive/RNAseq", "/content/drive/MyDrive/RNAseq/rawdata", "option"] {allow-input: true}

# %cd {DIR_RAWDATA}  # linux
os.chdir(DIR_WORK)
print("現在のディレクトリ:", os.getcwd())

# folder name
directories = ["bamfiles", "downloads", "featureCounts", "multiqc", "rawdata", "trimmed"]

# create folder function
def check_and_create_directory(directory):
    if not os.path.exists(directory):
        print(f"Creating directory: {directory}")
        os.makedirs(directory)
    else:
        print(f"Directory already exists: {directory}. Nothing to do.")

# create folder
for directory in directories:
    check_and_create_directory(directory)

###シーケンスリードデータ(.fastq.gz)とサンプルリストファイル(sample_list.txt)の準備

/content/drive/MyDrive/RNAseq/rawdata に  


* シーケンスリードデータ(.fastq.gz)
* サンプルリストファイル(sample_list.txt)  


を格納しておく。  
サンプルリストファイルは以下の通り。


| No. | ID  | Info |
|-----|-----|------|
| 1   | ABC | ...  |
| 2   | DEF | ...  |
| 3   | GHI | ...  |
| 4   | JKL | ...  |




In [ ]:
#@title コンテンツ表示
import os

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq" #@param ["/content", "/content/drive/MyDrive/RNAseq", "/content/drive/MyDrive/RNAseq/rawdata", "option"] {allow-input: true}

# ワーキングディレクトリの移動
os.chdir(DIR_WORK)
print("現在のディレクトリ:", os.getcwd())

# def print_directory_tree(directory, indent=''):
#     print(indent + os.path.basename(directory) + '/')
#     indent += '    '
#     for item in os.listdir(directory):
#         item_path = os.path.join(directory, item)
#         if os.path.isdir(item_path):
#             print_directory_tree(item_path, indent)
#         else:
#             print(indent + item)

# # 現在のワーキングディレクトリのツリー形式を表示
# print_directory_tree(os.getcwd())



## ----------シェルコマンドをGoogle Colaboratoryで実行----------
# !apt-get -qq install tree # tree表示のためのツールをインストール
!tree {DIR_WORK}
# !tree {DIR_RAWDATA}
## -------------------------------------------------------

In [ ]:
#@title サンプルリストの確認
import os
import pandas as pd

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
sample_list_file = "sample_list_original.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# データフレームに読み込む
df = pd.read_csv(sample_list_file, delimiter='\t')  # デリミターはタブに設定していますが、必要に応じて変更してください

# 最初の数行を表示
# print(df.head())
display(df.head().style)

# with open(sample_list_file, 'r') as file:
#     for i in range(5):  # 最初の5行を表示
#         line = file.readline()
#         print(line)

,No,SampleID,Analyte,Group,Drug,Drug1,Drug2
0,1,HIP_PC10,HIP,PC10,PCP_CLZ10,PCP,CLZ10
1,2,HIP_PC30,HIP,PC30,PCP_CLZ30,PCP,CLZ30
2,3,HIP_PV,HIP,PV,PCP_VEH,PCP,VEH
3,4,HIP_SC10,HIP,SC10,SAL_CLZ10,SAL,CLZ10
4,5,HIP_SC30,HIP,SC30,SAL_CLZ30,SAL,CLZ30


#トリミング(Trimmomatic)

In [ ]:
#@title ダウンロードトリミングツール: Trimmomatic
import os
import subprocess

# ディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_DOWNLOADS = "../downloads"  #@param ["../downloads", "/content/drive/MyDrive/RNAseq/downloads", "option1"] {allow-input: true}
DIR_TRIMMOMATIC = os.path.join(DIR_DOWNLOADS, "Trimmomatic-0.39")

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# すでにTrimmomaticが存在するかどうかをチェック
if not os.path.exists(DIR_TRIMMOMATIC):
    # TrimmomaticのZIPファイルをダウンロード
    trimmomatic_zip_url = "http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip"  #@param ["http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip", "option1", "option2"] {allow-input: true}
    trimmomatic_zip_file = os.path.basename(trimmomatic_zip_url)
    subprocess.run(["wget", trimmomatic_zip_url], cwd=DIR_DOWNLOADS)

    # TrimmomaticのZIPファイルを展開
    subprocess.run(["unzip", trimmomatic_zip_file], cwd=DIR_DOWNLOADS)
else:
    print("Trimmomaticはすでに展開されています。")


## ----------シェルコマンドをPythonコードで実行----------
# Trimmomaticをダウンロードして展開する
# %cd /content/drive/MyDrive/RNAseq/downloads
# !wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
# !unzip Trimmomatic-0.39.zip
## -------------------------------------------------------


In [ ]:
#@title トリミング（未実施サンプルデータのみ処理）: Trimmomatic
import os
import subprocess
import time

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
sample_list_file = "sample_list_original.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}
java_file = "../downloads/Trimmomatic-0.39/trimmomatic-0.39.jar"  #@param ["../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10", "option1", "option2"] {allow-input: true}
illuminaclip_file = "../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10"  #@param ["../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# サンプルリストをファイルから読み取る
with open(sample_list_file, "r") as f:
    sample_list = f.read().splitlines()

# サンプルリストをループで処理
for sample_line in sample_list[1:]:  # 1行目をスキップして残りの行を処理
    # サンプル名を分割して取得
    sample_data = sample_line.strip().split("\t")
    sample_number = sample_data[0]  # サンプル番号

    # 出力ファイル名を検査してトリミング済みかどうかを確認
    output_forward_paired = f"../trimmed/paired_{sample_data[1]}_1.trim.fastq.gz"
    output_reverse_paired = f"../trimmed/paired_{sample_data[1]}_2.trim.fastq.gz"
    if os.path.exists(output_forward_paired) and os.path.exists(output_reverse_paired):
        print(f"Sample No. {sample_number}, <{sample_data[1]}> has already been trimmed. Skipping...")
        continue

    # 入力ファイルのフルパスを指定
    input_forward = f"{sample_data[1]}_1.fastq.gz"
    input_reverse = f"{sample_data[1]}_2.fastq.gz"
    output_forward_paired = f"../trimmed/paired_{sample_data[1]}_1.trim.fastq.gz"
    output_forward_unpaired = f"../trimmed/unpaired_{sample_data[1]}_1.trim.fastq.gz"
    output_reverse_paired = f"../trimmed/paired_{sample_data[1]}_2.trim.fastq.gz"
    output_reverse_unpaired = f"../trimmed/unpaired_{sample_data[1]}_2.trim.fastq.gz"

    # ファイル名を出力して確認
    print(f"Sample No. {sample_number}, Forward Input File: {input_forward}, Reverse Input File: {input_reverse}")
    print(f"Sample No. {sample_number}, Forward Output File: {output_forward_paired}, {output_forward_unpaired}, Reverse Output File: {output_reverse_paired}, {output_reverse_unpaired}")

    # 処理開始時間を記録
    start_time = time.time()
    print(f"Processing started for sample {sample_data[1]} at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

    # コマンドを構築
    trimmomatic_cmd = [
        "java", "-jar", java_file, "PE", "-phred33",
        input_forward, input_reverse,
        output_forward_paired, output_forward_unpaired,
        output_reverse_paired, output_reverse_unpaired,
        f"ILLUMINACLIP:{illuminaclip_file}"
    ]

    # コマンド実行前のメッセージを表示
    print(f"Starting trimming for sample {sample_data[1]}...")

    # trimmomaticコマンドを実行
    result = subprocess.run(trimmomatic_cmd, capture_output=True, text=True)

    # 結果を表示
    print(f"Trimming for sample {sample_data[1]} finished with return code: {result.returncode}")
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)

    # 処理終了時間を記録
    end_time = time.time()

    # 処理時間を計算して表示
    processing_time = end_time - start_time
    print(f"Processing time for sample {sample_data[1]}: {processing_time} seconds")

    # 処理の区切りを示すメッセージを表示
    print("------------------------------------------------------")


#マッピング(HISAT2)

In [ ]:
#@title ゲノム配列情報（FNA）のダウンロード: HISAT2
import gzip
import os
import shutil
import subprocess

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_DOWNLOADS = "../downloads"  #@param ["../downloads", "/content/drive/MyDrive/RNAseq/downloads", "option1"] {allow-input: true}
fna_file = "GCF_000001635.27_GRCm39_genomic.fna"  #@param ["GCF_000001635.27_GRCm39_genomic.fna.gz", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# ファイルが存在するかチェック
if not os.path.exists(os.path.join(DIR_DOWNLOADS, fna_file)):
    # ダウンロード
    fna_file_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.fna.gz"  #@param ["https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.fna.gz", "option1", "option2"] {allow-input: true}
    subprocess.run(["wget", fna_file_url, "-P", DIR_DOWNLOADS])

    # 解凍
    fna_file_path = os.path.join(DIR_DOWNLOADS, fna_file + ".gz")
    output_file_path = os.path.join(DIR_DOWNLOADS, fna_file)
    with gzip.open(fna_file_path, 'rb') as f_in:
        with open(output_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
else:
    print("ファイルがすでに存在します。")


ファイルがすでに存在します。


In [ ]:
#@title （インストール: HISAT2）
# import os
# !apt-get -qq install hisat2
# !apt-get -qq install samtools
# os.environ['PATH'] += ":/usr/local/bin"
# !hisat2 --version
# !samtools --version

In [ ]:
#@title インデックスの作成: HISAT2
import os
import subprocess
import time

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
genomic_fna = "../downloads/GCF_000001635.27_GRCm39_genomic.fna"  #@param ["../downloads/GCF_000001635.27_GRCm39_genomic.fna", "option1", "option2"] {allow-input: true}
index_path = "../downloads/GCF_000001635.27_GRCm39_genomic_index"  #@param ["../downloads/GCF_000001635.27_GRCm39_genomic_index", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# 処理開始時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# インデックスファイルが存在するかチェック
if not os.path.exists(index_path + ".1.ht2"):
    # インデックスを作成
    print("Creating index...")
    subprocess.run(["hisat2-build", genomic_fna, index_path])
    print("Index creation completed.")
else:
    print("インデックスファイルはすでに存在します。")

# 処理終了時間を記録
end_time = time.time()

# 処理時間を計算して表示
processing_time = end_time - start_time
print(f"Processing time: {processing_time} seconds.")

# 2h 5m 2s


Creating index...
Index creation completed.


In [ ]:
#@title マッピング（未実施サンプルデータのみ処理）: HISAT2
import locale
import os
import subprocess
import time

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
sample_list_file = "sample_list_original.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}
index_path = "../downloads/GCF_000001635.27_GRCm39_genomic_index"  #@param ["../downloads/GCF_000001635.27_GRCm39_genomic_index", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# サンプルリストファイルからサンプルを取得し、HISAT2を実行
with open(sample_list_file, "r") as f:
    for line in f.readlines()[1:]:
        sample = line.strip().split()[1]
        print(f"Processing sample: {sample}...")
        file1 = f"../trimmed/paired_{sample}_1.trim.fastq.gz"
        file2 = f"../trimmed/paired_{sample}_2.trim.fastq.gz"
        bam_file = f"../bamfiles/{sample}_sorted.bam"

        # 処理開始時間を記録
        start_time = time.time()
        print(f"Processing started for sample {sample} at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

        # すでに処理されたサンプルかどうかを確認
        if os.path.exists(bam_file):
            print(f"Sample {sample} has already been processed. Skipping...")
            continue

        # HISAT2を実行
        hisat2_cmd = [
            "hisat2", "-p", "1", "-x", index_path, "-1", file1, "-2", file2
        ]
        hisat2_proc = subprocess.Popen(hisat2_cmd, stdout=subprocess.PIPE)

        # samtoolsを使ってバムファイルに変換
        samtools_cmd = [
            "samtools", "sort", "-@ 1", "-", "-o", bam_file
        ]
        samtools_proc = subprocess.Popen(samtools_cmd, stdin=hisat2_proc.stdout)
        samtools_proc.communicate()

        # バムファイルにインデックスを作成
        subprocess.run(["samtools", "index", "-@", "1", bam_file])

        # 処理終了時間を記録
        end_time = time.time()

        # 処理時間を計算して表示
        processing_time = end_time - start_time
        print(f"Sample {sample} processing complete. Processing time: {processing_time} seconds.")


#カウント(featureCount(subread))

In [ ]:
#@title アノテーション（GTF）のダウンロード: featureCount(subread)
import gzip
import os
import shutil
import subprocess

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_DOWNLOADS = "../downloads"  #@param ["../download", "/content/drive/MyDrive/RNAseq/downloads", "option1"] {allow-input: true}
gtf_file = "GCF_000001635.27_GRCm39_genomic.gtf"  #@param ["GCF_000001635.27_GRCm39_genomic.gtf", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# ファイルが存在するかチェック
if not os.path.exists(os.path.join(DIR_DOWNLOADS, gtf_file)):
    # ダウンロード
    gtf_file_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.gtf.gz"  #@param ["https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.gtf.gz", "option1", "option2"] {allow-input: true}
    subprocess.run(["wget", gtf_file_url, "-P", DIR_DOWNLOADS])

    # 解凍
    gtf_file_path = os.path.join(DIR_DOWNLOADS, gtf_file + ".gz")
    output_file_path = os.path.join(DIR_DOWNLOADS, gtf_file)
    with gzip.open(gtf_file_path, 'rb') as f_in:
        with open(output_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
else:
    print("ファイルがすでに存在します。")

ファイルがすでに存在します。


In [ ]:
#@title リードカウント: featureCount(subread)
import os
import subprocess
import time
!apt-get -qq install subread
os.environ['PATH'] += ":/usr/local/bin"

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
os.chdir(DIR_WORK)

# ファイルパス
annotation_file = "/content/drive/MyDrive/RNAseq/downloads/GCF_000001635.27_GRCm39_genomic.gtf"  #@param ["/content/drive/MyDrive/RNAseq/downloads/GCF_000001635.27_GRCm39_genomic.gtf", "../downloads/GCF_000001635.27_GRCm39_genomic.gtf", "option1", "option2"] {allow-input: true}
input_file = "/content/drive/MyDrive/RNAseq/bamfiles/*_sorted.bam" #@param ["/content/drive/MyDrive/RNAseq/bamfiles/*_sorted.bam", "../bamfiles/*_sorted.bam", "option1", "option2"] {allow-input: true}
output_file = "/content/drive/MyDrive/RNAseq/featureCounts/featureCounts.txt"  #@param ["/content/drive/MyDrive/RNAseq/featureCounts/featureCounts.txt", "../featureCounts/featureCounts.txt", "option1", "option2"] {allow-input: true}

# featureCountsの定義
# featureCounts_cmd = [
#     "featureCounts",
#     "-F", "GTF",
#     "-t", "CDS",
#     "-g", "gene_id",
#     "-a", annotation_file,
#     "-o", output_file,
#     "-p", input_file
# ]

# 実行時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# featureCountsを実行
# subprocess.run(featureCounts_cmd)
!featureCounts -F GTF -t CDS -g gene_id \
    -a $annotation_file \
    -o $output_file \
    -p $input_file

# 実行時間を計算して表示
end_time = time.time()
execution_time = end_time - start_time
print(f"Processing time: {execution_time} seconds.")


#その他

###(seqkit: 実装できていない)

In [ ]:
#@title (seqkit: 実装できていない)
# %cd /content

# !git clone https://github.com/shenwei356/seqkit.git
# %cd seqkit
# !make
# !sudo make install

In [ ]:
## ----------シェルコマンドをPythonコードで実行----------
# Trimmomaticをダウンロードして展開する
# %cd /content/drive/MyDrive/RNAseq/downloads
# !wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
# !unzip Trimmomatic-0.39.zip
## -------------------------------------------------------

###(トリミング: 実行実績あり: Trimmomatic)

In [ ]:
#(トリミング: 実行実績あり: Trimmomatic)

import os
import subprocess
import time

# ファイルパス
%cd /content/drive/MyDrive/RNAseq/rawdata
sample_list_file = "sample_list.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}
ILLUMINACLIP_file = "../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10"  #@param ["../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10", "option1", "option2"] {allow-input: true}
JAVA_file = "../downloads/Trimmomatic-0.39/trimmomatic-0.39.jar"  #@param ["../downloads/Trimmomatic-0.39/trimmomatic-0.39.jar", "option1", "option2"] {allow-input: true}

# サンプルリストをファイルから読み取る
with open(sample_list_file, "r") as f:
    sample_list = f.read().splitlines()

# サンプルリストをループで処理
for sample_line in sample_list[1:]:  # 1行目をスキップして残りの行を処理
    # サンプル名を分割して取得
    sample_data = sample_line.strip().split("\t")
    sample = sample_data[0]  # サンプル名
    # 入力ファイルのフルパスを指定
    input_forward = f"{sample_data[1]}_1.fastq.gz"
    input_reverse = f"{sample_data[1]}_2.fastq.gz"
    output_forward = f"../trimmed/paired_{sample_data[1]}_1.trim.fastq.gz ../trimmed/unpaired_{sample_data[1]}_1.trim.fastq.gz "
    output_reverse = f"../trimmed/paired_{sample_data[1]}_2.trim.fastq.gz ../trimmed/unpaired_{sample_data[1]}_2.trim.fastq.gz "

    # ファイル名を出力して確認
    print(f"Sample: {sample}, Forward Input File: {input_forward}, Reverse Input File: {input_reverse}")
    print(f"Sample: {sample}, Forward Output File: {output_forward}, Reverse Output File: {output_reverse}")

    # 処理開始時間を記録
    start_time = time.time()

    # コマンドを構築
    trimmomatic_cmd = f"java -jar {JAVA_file} PE -phred33 {input_forward} {input_reverse} " \
                          f"{output_forward} {output_reverse} " \
                          f"ILLUMINACLIP:{ILLUMINACLIP_file}"

    # コマンド実行前のメッセージを表示
    print(f"Starting trimming for sample {sample_data[1]}...")

    # trimmomaticコマンドを実行
    result = subprocess.run(trimmomatic_cmd, shell=True, capture_output=True, text=True)

    # 結果を表示
    print(f"Trimming for sample {sample_data[1]} finished with return code: {result.returncode}")
    print(result.stdout)
    print(result.stderr)

    # 処理終了時間を記録
    end_time = time.time()

    # 処理時間を計算して表示
    processing_time = end_time - start_time
    print(f"Processing time for sample {sample_data[1]}: {processing_time} seconds")

    # 処理の区切りを示すメッセージを表示
    print(f"------------------------------------------------------")


###(クオリティコントロールレポート(MultiQC): リポートが出力されない)

In [ ]:
import os

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content"  #@param ["/content", "/content/drive/MyDrive/RNAseq/rawdata", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

!git clone https://github.com/MultiQC/MultiQC_Notebook.git



In [ ]:
# %pip install --force-reinstall --upgrade git+https://github.com/ewels/MultiQC.git

In [ ]:
import multiqc
print(multiqc.__version__)

1.22.dev0


In [ ]:
# (クオリティコントロールレポート: MultiQC): リポートが出力されない
# %pip install --force-reinstall --upgrade git+https://github.com/ewels/MultiQC.git

import os
import subprocess
import time
import multiqc
print(multiqc.__version__)

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_MULTIQC = "/content/drive/MyDrive/RNAseq/multiqc"  #@param ["../multiqc", "/content/drive/MyDrive/RNAseq/multiqc", "option1"] {allow-input: true}

# 入力ディレクトリのリスト
input_path_1 = "/content/drive/MyDrive/RNAseq/rawdata"  #@param [".", "/content/drive/MyDrive/RNAseq/rawdata", "option1"] {allow-input: true}
input_path_2 = "/content/drive/MyDrive/RNAseq/trimmed"  #@param ["../trimmed", "/content/drive/MyDrive/RNAseq/trimmed", "option1"] {allow-input: true}
input_path_3 = ""  #@param ["", "option1", "option2"] {allow-input: true}

DIR_INPUT = [input_path_1, input_path_2, input_path_3]
DIR_ANALYSIS = DIR_INPUT

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# 処理開始時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# MultiQCを実行
multiqc.run(
            # input_dir= DIR_INPUT,
            output_dir=DIR_MULTIQC,
            analysis_dir=DIR_ANALYSIS)

print("MultiQC processing completed.")

# 処理終了時間を記録
end_time = time.time()

# 処理時間を計算して表示
processing_time = end_time - start_time
print(f"Processing time: {processing_time} seconds.")


In [ ]:
import IPython

# Best if using Google Colab
IPython.display.HTML(filename='./multiqc_report.html')

# Best if running locally
# IPython.display.IFrame('./multiqc_report.html', '100%', 600)

###(フォルダとファイルの削除→必要があればアクティブにして実行)

In [ ]:
# import os
# import shutil

# 削除するディレクトリの親ディレクトリのパス
# parent_directory = "/content/drive/MyDrive/RNAseq" #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}

# 削除するサブディレクトリのリスト
# subdirectories = ["downloads", "trimmed", "bamfiles", "featureCounts", "fastp", "fastp.1", "seqkit"]

# サブディレクトリとファイルを削除
# for subdir in subdirectories:
#     path = os.path.join(parent_directory, subdir)
#     if os.path.isdir(path):
#         os.rmdir(path)
#         print(f"{path} を削除しました。")
#     elif os.path.isfile(path):
#         os.remove(path)
#         print(f"{path} を削除しました。")
#     else:
#         print(f"{path} は存在しません。")

# サブディレクトリを再帰的に削除
# for subdir in subdirectories:
#     path = os.path.join(parent_directory, subdir)
#     if os.path.exists(path):
#         if os.path.isdir(path):
#             shutil.rmtree(path)
#             print(f"{path} を再帰的に削除しました。")
#         else:
#             print(f"{path} はディレクトリではないため削除できません。")
#     else:
#         print(f"{path} は存在しません。")

/content/drive/MyDrive/RNAseq/downloads は存在しません。
/content/drive/MyDrive/RNAseq/trimmed は存在しません。
/content/drive/MyDrive/RNAseq/bamfiles は存在しません。
/content/drive/MyDrive/RNAseq/featureCounts は存在しません。
/content/drive/MyDrive/RNAseq/fastp は存在しません。
/content/drive/MyDrive/RNAseq/fastp.1 は存在しません。
/content/drive/MyDrive/RNAseq/seqkit を再帰的に削除しました。
